In [2]:
from __future__ import unicode_literals

from mongoengine import *
from ConfigParser import SafeConfigParser
import matplotlib
import matplotlib.pyplot as plt
from processing import *
from operator import itemgetter
from itertools import groupby
from sklearn import cluster
from sklearn.metrics.pairwise import cosine_similarity

# Reading Config file
config = SafeConfigParser()
config.read('config.ini')

# Initialisation de la connexion  a la BDD
connect(config.get('database', 'name'))

MongoClient('localhost', 27017)

In [9]:
table = []
for article in Article.objects:
    table.append((article,article.num_cluster))

sorted_table = sorted(table, key=itemgetter(1))
table = groupby(sorted_table, key=itemgetter(1))
articles = [{'cluster':k,
                    'articles':[elms1 for (elms1,elms2) in v]} for k, v in table]

event = {}
for ind,value in enumerate(articles):
    event[ind] = []
    for article in articles[ind]['articles']:
        event[ind].append(article.pub_date)
        

In [12]:
for cluster,date_event in event.iteritems():
    print(date_event)

[u'2017-05-19', u'2017-05-18', u'2017-05-19', u'2017-05-24', u'2017-05-24', u'2017-05-23', u'2017-05-28', u'2017-05-26', u'2017-05-30', u'2017-05-30', u'2017-05-30']
[u'2017-05-19', u'2017-05-19', u'2017-05-19', u'2017-05-18', u'None', u'None', u'None', u'2017-05-23', u'2017-05-23', u'0974-05-29', u'2017-05-28', u'None']
[u'2017-05-19', u'2017-05-19', u'0974-05-23', u'2017-05-30']
[u'0974-05-19', u'0974-05-19', u'0974-05-19', u'2017-05-19', u'2017-05-19', u'0974-05-19', u'2017-05-19', u'2017-05-19', u'2017-05-18', u'None', u'2017-05-23', u'2017-05-26', u'2017-05-22', u'2017-05-26', u'None', u'2017-05-20', u'2017-05-25', u'2017-05-30', u'2017-05-30']
[u'2017-05-19', u'None', u'None']
[u'2017-05-19', u'2017-05-19', u'2017-05-19', u'2017-05-16', u'2017-05-18', u'2017-05-12', u'2017-05-15', u'None', u'2017-05-22', u'2017-05-23', u'2017-05-22', u'2017-05-22', u'2017-05-22', u'2017-05-24', u'2017-05-23', u'2017-05-23', u'2017-05-23', u'2017-05-29', u'2017-05-29', u'2017-05-30']
[u'2017-05-12

table = []
for article in Article.objects:
    table.append((article,article.num_cluster))

sorted_table = sorted(table, key=itemgetter(1))
table = groupby(sorted_table, key=itemgetter(1))
articles = [{'cluster':k,
                    'articles':[elms1 for (elms1,elms2) in v]} for k, v in table]

x = [cluster for cluster,title in enumerate(articles)]
y = [len(articles[ind]['articles']) for ind,value in enumerate(articles)]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x, y)

plt.xlabel('liste des clusters')
plt.ylabel('nombre d\'articles par cluster' )

plt.title("Répartition des articles par cluster")
plt.show()

table = [article.num_cluster for article in Article.objects]
N = len(table)
x = np.random.rand(N)
y = np.random.rand(N)
cluster = np.array(table)
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(x,y,c=cluster,s=20)
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(scatter)
plt.show()

def mean_shift_clusterize(articles):
    length_clusters = {}
    for article in articles:
        articles_texts = []
        for each_article in article['articles']:
            articles_texts.append(each_article.text)

        
        vectorizer = TfidfVectorizer(max_features=200000)
        text_matrix = vectorizer.fit_transform(articles_texts)
        dist = 1 - cosine_similarity(text_matrix)
        
        #bandwidth = cluster.estimate_bandwidth(dist, quantile=0.3)
        #mean_shift_classifier = cluster.MeanShift(bandwidth=bandwidth)
        mean_shift_classifier = cluster.MeanShift()
                
        try :
            mean_shift_classifier.fit(dist)
        except ValueError:
            print('Too close bandwidth')
            length_clusters[article['cluster']] = 0
            continue
            
        cluster_labels = mean_shift_classifier.labels_
        cluster_centers = mean_shift_classifier.cluster_centers_
        
        #gp_clusters = [{'cluster':value,
        #'article_id':content.keys()[idx]} for (idx,vlue) in enumerate(clusters)]

        length_clusters[article['cluster']] = len(cluster_centers)
    return length_clusters 
        